In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation 

In [2]:
num_var = 3

In [3]:
def runge_kutta_1(f,tspan,y0,h):
    x = np.linspace(tspan[0],tspan[1],int((tspan[1]- tspan[0])/h))
    y = np.zeros((len(x),num_var))
    y[0,:] = y0
    for i in range(len(x)-1):
        k_1 = np.array(f(x[i],y[i,:]))
        y[i+1,:] = y[i,:] + k_1*h
    return x,y

In [4]:
def runge_kutta_2(f,tspan,y0,h):
    x = np.linspace(tspan[0],tspan[1],int((tspan[1]- tspan[0])/h))
    y = np.zeros((len(x),3))
    y[0,:] = y0
    for i in range(len(x)-1):
        k_1 = np.array(f(x[i],y[i,:]))
        k_2 = np.array(f(x[i] + h,  y[i,:] + h*k_1))
        y[i+1,:] = y[i,:] + (1/2)*(k_1 + k_2)*h
    return x,y

In [5]:
def runge_kutta_3(f,tspan,y0,h):
    x = np.linspace(tspan[0],tspan[1],int((tspan[1]- tspan[0])/h))
    y = np.zeros((len(x),3))
    y[0,:] = y0
    for i in range(len(x)-1):
        k_1 = np.array(f(x[i],y[i,:]))
        k_2 = np.array(f(x[i] + 0.5*h,  y[i,:] + 0.5*h*k_1))
        k_3 = np.array(f((x[i] + h),  (y[i,:] - h*k_1 + 2*h*k_2)))
        y[i+1,:] = y[i,:] + (1/6)*(k_1+4*k_2+k_3)*h
    return x,y

In [6]:
def runge_kutta_4(f,tspan,y0,h):
    x = np.linspace(tspan[0],tspan[1],int((tspan[1]- tspan[0])/h))
    y = np.zeros((len(x),num_var))
    y[0,:] = y0
    for i in range(len(x)-1):
        k_1 = np.array(f(x[i],y[i,:]))
        k_2 = np.array(f(x[i]+0.5*h,  y[i,:]+0.5*h*k_1))
        k_3 = np.array(f((x[i]+0.5*h),  (y[i,:]+0.5*h*k_2)))
        k_4 = np.array(f((x[i]+h),  (y[i,:]+k_3*h)))
        y[i+1,:] = y[i,:] + (1/6)*(k_1+2*k_2+2*k_3+k_4)*h
    return x,y

In [8]:
def Find_u(z,alpha):
    b = z[0]
    c = z[1]
    
    x_start = 0
    x_end = 1
    x = x_start
    
    u = 0
    v = alpha
    
    N = 1000
    delta = (x_end-x_start)/N
    for i in range(N):
        u = u + delta*v
        v = v + delta*(c/b*v - 1/b *((math.pi)**2*(math.sin((math.pi)*x)) + math.pi*math.cos((math.pi)*x)))
        x = x + delta
    u1 = u
    return u1

In [9]:
from sympy import symbols, solve

def initialvalue(z):
    x = symbols('x')
    f = Find_u(z,x)
    sol = solve(f)
    return sol

In [18]:
def SOL_PDE_UV(z):
    b = z[0]
    c = z[1]
    
    x_start = 0
    x_end = 1
    x = x_start
    alpha = initialvalue(z)[0]
    
    U = []
    V = []
    u = 0
    v = alpha
    
    N = 1000
    delta = (x_end-x_start)/N
    for i in range(N):
        U.append(u)
        V.append(v)
        u = u + delta*v
        v = v + delta*(c/b*v - (math.pi**2*math.sin(math.pi*x) + math.pi*math.cos(math.pi*x))/b)
        x = x + delta
    return U,V

In [21]:
def Find_p(z,alpha):
    b = z[0]
    c = z[1]
    
    U,V = SOL_PDE_UV([1,1])
    
    x_start = 0
    x_end = 1
    x = x_start
    
    p = 0
    q = alpha
    
    N = 1000
    delta = (x_end-x_start)/N
    for i in range(N):
        p = p + delta*q
        q = q + delta*(-c/b*q - (U[i] - math.sin(math.pi*x))/b)
        x = x + delta
    p1 = p
    return p1

In [22]:
def initialvalue_p(z):
    x = symbols('x')
    f = Find_p(z,x)
    sol = solve(f)
    return sol

In [26]:
def SOL_PDE_PQ(z):
    b = z[0]
    c = z[1]
    
    x_start = 0
    x_end = 1
    x = x_start
    alpha = initialvalue_p(z)[0]
    U,V = SOL_PDE_UV(z)
    
    P = []
    Q = []
    p = 0
    q = alpha
    
    N = 1000
    delta = (x_end-x_start)/N
    for i in range(N):
        P.append(p)
        Q.append(q)
        p = p + delta*q
        q = q + delta*(-c/b*q - (U[i] - math.sin(math.pi*x))/b)
        x = x + delta
    return P,Q

In [30]:
def gradientJ(z):
    U,V = SOL_PDE_UV(z)
    P,Q = SOL_PDE_PQ(z)
    
    N = len(U)
    barU = np.array(U)
    barV = np.array(V)
    barP = np.array(P)
    barQ = np.array(Q)
    
    gradient_b = -1/N* sum(barV*barQ)
    gradient_c = -1/N* sum(barV*barP)
    gradient = np.array([gradient_b,gradient_c],dtype = np.float64)
    return gradient

In [37]:
def GD(gradientJ, z0, eta = 0.1):
    z = [z0]
    z_new = z[-1] - eta*gradientJ(z[-1])
    z.append(z_new)
    for it in range(50):
        z_new = z[-1] - eta*gradientJ(z[-1])
        norm2 = sum(z_new*z_new)
        if math.sqrt(norm2) < 1e-4:
            break
        z.append(z_new)
    return z[-1]

In [39]:
GD(gradientJ, np.array([0.98,0.98]), eta = 0.1)

array([0.99718137, 0.98783732])